In [ ]:
# pip install langchain langchain-community langchain-experimental transformers accelerate torch pillow


In [ ]:
# pip install bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline


In [ ]:
from google.colab import drive
drive.flush_and_unmount() # Disconnects any existing link
drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

save_dir = "/content/drive/MyDrive/models/mistral7b_4bit"

# after saving it load directly from your drive

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)

# tokenizer = AutoTokenizer.from_pretrained(save_dir)

# # Load the model with quantization
# model = AutoModelForCausalLM.from_pretrained(
#     save_dir,
#     quantization_config=bnb_config,
#     device_map="auto"
# )



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Pipeline for routing / supervision
router_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,     # routing should be SHORT
    temperature=0.0,        # deterministic
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id
)

# LangChain-compatible LLM
router_llm = HuggingFacePipeline(pipeline=router_pipe)

Device set to use cuda:0


SQL tool

In [ ]:
sql_model_id = "defog/sqlcoder-7b-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


sql_tokenizer = AutoTokenizer.from_pretrained(sql_model_id)

sql_model = AutoModelForCausalLM.from_pretrained(
    sql_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto"
)

sql_tokenizer.save_pretrained(save_dir)
sql_model.save_pretrained(save_dir)

# sql_tokenizer = AutoTokenizer.from_pretrained(save_dir)

# sql_model = AutoModelForCausalLM.from_pretrained(
#     save_dir,
#     quantization_config=bnb_config,
#     device_map="auto",
#     torch_dtype="auto"
# )

sql_pipe = pipeline(
    "text-generation",
    model=sql_model,
    tokenizer=sql_tokenizer,
    max_new_tokens=512,
    temperature=0,
    do_sample=True
)

sql_llm = HuggingFacePipeline(pipeline=sql_pipe)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-309363961.py:32: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  sql_llm = HuggingFacePipeline(pipeline=sql_pipe)


SQL database chain and tool wrapper

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# Load CSV
csv_path = "/content/cdac_timestamp_power_deviation_dataset.csv"
df = pd.read_csv(csv_path)

# Create SQLite DB
conn = sqlite3.connect("power_data.db")

# Write CSV to SQL table
df.to_sql(
    name="power_data",
    con=conn,
    if_exists="replace",
    index=False
)

conn.close()


In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///power_data.db")

sql_chain = SQLDatabaseChain.from_llm(
    llm=sql_llm,
    db=db,
    verbose=True,
    return_direct=True
)

In [ ]:
from langchain_core.tools import Tool

sql_tool = Tool(
    name="sql_query_tool",
    description=(
        "Use this tool when the question requires querying structured data, "
        "aggregations, trends over time, before/after event analysis."
    ),
    func=sql_chain.run
)

Vision tool

In [ ]:
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from PIL import Image

vision_model_id = "Qwen/Qwen2-VL-7B-Instruct"

vision_processor = AutoProcessor.from_pretrained(vision_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

vision_model = Qwen2VLForConditionalGeneration.from_pretrained(
    vision_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto"
)

def analyze_graph_with_qwen(query: str, image_path: str = "graph_ma.png"):
    image = Image.open(image_path).convert("RGB")

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": query}
            ]
        }
    ]

    inputs = vision_processor(
        messages,
        return_tensors="pt"
    ).to(vision_model.device)

    with torch.no_grad():
        output = vision_model.generate(
            **inputs,
            max_new_tokens=512
        )

    return vision_processor.batch_decode(
        output, skip_special_tokens=True
    )[0]


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

vision tool wrapper (custom wrapper)

In [ ]:
vision_tool = Tool(
    name="graph_insight_tool",
    description=(
        "Use this tool when the user asks to analyze charts, graphs, "
        "visual trends, or insights visible in an image."
    ),
    func=lambda q: analyze_graph_with_qwen(q, "graph_ma.png")
)


Agent (decision maker)

In [ ]:
# from langgraph.prebuilt import create_react_agent

# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# tools = [sql_tool, vision_tool]

# # Define the prompt template for the ReAct agent
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful assistant."),
#         ("human", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ]
# )

# # Create the agent
# agent = create_react_agent(
#     model=router_llm,
#     tools=tools,
# )

# for event in agent.stream(
#     {"input": "Analyze the graph and query the DB if needed"}
# ):
#     print(event)

In [ ]:
from langchain_core.prompts import PromptTemplate


In [ ]:
router_prompt = PromptTemplate(
    input_variables=["query"],
    template="""
You are a routing agent.

Decide which agent should handle the user query.

Return ONLY one of:
- sql
- vision

Rules:
- Use sql if the question requires querying structured data, aggregations, trends, or before/after analysis
- Use vision if the question requires understanding a chart, image, or graph

User query:
{query}
"""
)


In [ ]:
# !pip install langchain-classic

In [ ]:
from langchain_classic.agents import initialize_agent, AgentType

In [ ]:
agent = initialize_agent(
    tools=[sql_tool, vision_tool],
    llm=router_llm,   # Mistral HF pipeline
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

NameError: name 'sql_tool' is not defined

In [ ]:
# model = router_llm
# chain = router_prompt | model

In [ ]:
query = (
    "After painting event on 28 August 2025, "
    "did power deviation reduce and by how much?"
)
response = agent.run(query)

print(response)


You are a routing agent.

Decide which agent should handle the user query.

Return ONLY one of:
- sql
- vision
- chat

Rules:
- Use sql if the question requires querying structured data, aggregations, trends, or before/after analysis
- Use vision if the question requires understanding a chart, image, or graph
- Use chat otherwise

User query:
After painting event on 28 August 2025, did power deviation reduce and by how much?

Answer: sql
